<a href="https://colab.research.google.com/github/vmaeeeee03/CompLing_2024/blob/main/SMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Статистический машинный перевод (SMT)**

Статистический машинный перевод (Statistical Machine Translation, SMT) представляет собой метод машинного перевода, основанный на статистических моделях, которые формируются на основе анализа двуязычных корпусов текста. Этот подход противопоставляется системам, основанным на правилах (Rule-Based Machine Translation, RBMT) и на примерах (Example-Based MT, EBMT).

**Основные компоненты SMT**
Языковая модель (Language Model): использует n-граммы для моделирования вероятностного распределения слов или фраз в целевом языке, оценивает грамматичность последовательностей слов.

Модель перевода (Translation Model): собирает статистику соответствий фраз из параллельного корпуса, определяя вероятности перевода для пар последовательностей.

Декодер (Decoder): определяет наиболее грамматически правильные и лексически подходящие варианты перевода из множества гипотез.

**Преимущества и недостатки SMT**
Преимущества:

*   Быстрая настройка и возможность добавления новых языковых направлений.
*   Высокая точность переводов для фраз, полностью помещающихся в n-граммную модель.

Недостатки:

*   Зависимость от наличия параллельных корпусов, что может приводить к дефициту данных.
*   Склонность к грамматическим ошибкам и нестабильности переводов.


**Этапы работы SMT**
1. Сбор параллельных корпусов: Необходимость больших коллекций текстов с переводами.
2. Обучение моделей: Формирование вероятностных моделей на основе собранных данных.
3. Перевод: Генерация наиболее вероятных переводов для новых текстов.
4. Декодирование: Поиск наиболее подходящих переводов с учетом статистических оценок.
5. Постобработка: Учет особенностей языков для получения связного текста.


# **Импорт библиотек**

In [ ]:
import tarfile

from sklearn.model_selection import train_test_split

from collections import Counter, defaultdict
import random

# **Препроцессинг**

In [ ]:
# извлекаем файлы из архива
with tarfile.open('German English Tiny.tgz', 'r:gz') as tar:
  tar.extractall()

In [ ]:
!ls

 de-en.tiny.de	 de-en.tiny.en	'German English Tiny.tgz'   sample_data


In [ ]:
with open('de-en.tiny.de', 'r') as f:
  german = f.read().split('\n')[:-1]

with open('de-en.tiny.en', 'r') as f:
  english = f.read().split('\n')[:-1]

print("Данные языка X:\n", german)
print("Данные языка Y:\n", english)

Данные языка X:
 ['Wiederaufnahme der Sitzungsperiode', 'Ich bitte Sie , sich zu einer Schweigeminute zu erheben .', '( Das Parlament erhebt sich zu einer Schweigeminute . )', 'Frau Präsidentin , zur Geschäftsordnung .', 'Wenn das Haus damit einverstanden ist , werde ich dem Vorschlag von Herrn Evans folgen .', 'Frau Präsidentin , zur Geschäftsordnung .', 'Könnten Sie mir eine Auskunft zu Artikel 143 im Zusammenhang mit der Unzulässigkeit geben ?', 'Und zwar sollen derartige Strafen trotz des Grundsatzes der relativen Stabilität verhängt werden .', 'All dies entspricht den Grundsätzen , die wir stets verteidigt haben .', 'Vielen Dank , Herr Segni , das will ich gerne tun .', 'Das ist ganz im Sinne der Position , die wir als Parlament immer vertreten haben .', 'Das ist der Fall von Alexander Nikitin .', 'Nun ist es aber so , daß er wieder angeklagt werden soll , weil der Staatsanwalt in Berufung geht .', 'Dennoch , Frau Präsidentin , wurde meinem Wunsch nicht entsprochen .', 'Deshalb mö

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(english, german)

print("> Обучающая выборка:")
for text, label in zip(X_train, y_train):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

print("> Тестовая выборка:")
for text, label in zip(X_test, y_test):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

> Обучающая выборка:

Текст на немецком: Sollten wir nicht eigentlich eher darüber diskutieren ?
 Его перевод на английский: Should we not be discussing this issue ?


Текст на немецком: Letztlich können Beihilfen dazu dienen , Umstrukturierungen zu ermöglichen , Schulungen anzubieten , Arbeitsplätze und damit Know-how zu retten .
 Его перевод на английский: Finally , aid can enable restructuring , offer training , save jobs and thus know-how .


Текст на немецком: Herr Wynn , das ist logisch .
 Его перевод на английский: Mr Wynn , that makes sense .


Текст на немецком: Im Prinzip sind nur horizontale Regelungen zulässig , weil sie den Wettbewerb nicht oder kaum verzerren .
 Его перевод на английский: In principle , only horizontal regulations are permissible because they do not distort , or hardly distort , competition .


Текст на немецком: Wir haben diskutiert , wir waren uns einig , nur die EVP-Fraktion und die Liberalen nicht .
 Его перевод на английский: We discussed that matter

# **Подготовка данных**

In [ ]:
def tokenize(sentences):
  # функция возвращает списки слов
  return [sentence.split() for sentence in sentences]

# токенизируем каждую выборку
X_train_tokens, X_test_tokens, y_train_tokens, y_test_tokens = tokenize(X_train), tokenize(X_test), tokenize(y_train), tokenize(y_test)

print('Образец токенизированного текста:', X_train_tokens)

Образец токенизированного текста: [['Should', 'we', 'not', 'be', 'discussing', 'this', 'issue', '?'], ['Finally', ',', 'aid', 'can', 'enable', 'restructuring', ',', 'offer', 'training', ',', 'save', 'jobs', 'and', 'thus', 'know-how', '.'], ['Mr', 'Wynn', ',', 'that', 'makes', 'sense', '.'], ['In', 'principle', ',', 'only', 'horizontal', 'regulations', 'are', 'permissible', 'because', 'they', 'do', 'not', 'distort', ',', 'or', 'hardly', 'distort', ',', 'competition', '.'], ['We', 'discussed', 'that', 'matter', 'and', 'we', 'were', 'unanimous', ',', 'with', 'the', 'exception', 'of', 'the', 'PPE', 'and', 'ELDR', 'Groups', '.'], ['This', 'lack', 'of', 'infrastructure', 'is', 'also', 'an', 'obstacle', 'to', 'the', 'establishment', 'of', 'companies', 'and', 'the', 'creation', 'of', 'jobs', '.'], ['Safety', 'advisers', 'for', 'the', 'transport', 'of', 'dangerous', 'goods'], ['Subsidies', 'are', 'not', 'enough', 'to', 'ensure', 'development', 'when', 'infrastructure', 'and', 'public', 'service

In [ ]:
x_vocab = Counter(' '.join(french).split()).keys()
y_vocab = Counter(' '.join(english).split()).keys()

print(f"Словарь немецких словоформ: {x_vocab}\n Всего {len(x_vocab)} словоформ\n")
print(f"Cловарь английских словоформ: {y_vocab}\n Всего {len(y_vocab)} словоформ")

Словарь немецких словоформ: dict_keys(['Reprise', 'de', 'la', 'session', 'Je', 'vous', 'invite', 'à', 'lever', 'pour', 'cette', 'minute', 'silence', '.', '(', 'Le', 'Parlement', ',', 'debout', 'observe', 'une', ')', 'Madame', 'Présidente', "c'", 'est', 'motion', 'procédure', 'Si', "l'", 'Assemblée', 'en', "d'", 'accord', 'je', 'ferai', 'comme', 'M.', 'Evans', 'a', 'suggéré', 'voudrais', 'demander', 'un', 'conseil', 'au', 'sujet', 'article', '143', 'qui', 'concerne', 'irrecevabilité', 'Il', 'précise', 'que', 'cela', 'devrait', 'être', 'fait', 'malgré', 'le', 'principe', 'stabilité', 'relative', 'Et', 'tout', 'ceci', 'dans', 'respect', 'des', 'principes', 'nous', 'avons', 'toujours', 'soutenus', 'Merci', 'Monsieur', 'Segni', 'bien', 'volontiers', "C'", 'effet', 'ligne', 'positions', 'notre', 'adoptées', "s'", 'agit', 'du', 'cas', 'Alexandre', 'Nikitin', 'Toutefois', 'ce', "j'", 'avais', 'demandé', "n'", 'pas', 'été', 'réalisé', 'demande', 'donc', 'nouveau', 'faire', 'nécessaire', 'puissi

# **Модель SMT**

In [ ]:
# вероятность того, что случайное слово x_vocab соответсвует случайному слову y_vocab
uniform = 1 / (len(x_vocab) * len(y_vocab))

round(uniform, 3)

0.0

In [ ]:
# t-model
t = {}

for i in range(len(X_train)):
  # начинаем итерацию по обучающей выборке
  for word_x in X_train_tokens[i]:
    for word_y in y_train_tokens[i]:
      # создаем t-table
      t[(word_x, word_y)] = uniform

# t-table
for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | with   ->   außerordentlich | Вероятность: 0.0
Соответствие | with   ->   begrüßen | Вероятность: 0.0
Соответствие | the   ->   Subsidiarität | Вероятность: 0.0
Соответствие | the   ->   außerordentlich | Вероятность: 0.0
Соответствие | the   ->   begrüßen | Вероятность: 0.0
Соответствие | principle   ->   All | Вероятность: 0.0
Соответствие | principle   ->   dies | Вероятность: 0.0
Соответствие | principle   ->   ist | Вероятность: 0.0
Соответствие | principle   ->   im | Вероятность: 0.0
Соответствие | principle   ->   Sinne | Вероятность: 0.0
Соответствие | principle   ->   der | Вероятность: 0.0
Соответствие | principle   ->   Subsidiarität | Вероятность: 0.0
Соответствие | principle   ->   außerordentlich | Вероятность: 0.0
Соответствие | principle   ->   zu | Вероятность: 0.0
Соответствие | principle   ->   begrüßen | Вероятность: 0.0
Соответствие | subsidiarity   ->   All | Вероятность: 0.0
Соотв

In [ ]:
# количество итераций обучения
epochs = 7

In [ ]:
for epoch in range(epochs):
  # начинаем обучение

  # шаг 0. создаем слоты для подсчета статистики
  count = {} # P(x|y)
  total = {} # P(y)

  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # создаем слоты для подсчета условной вероятности совпадений в корпусе
        count[(word_x, word_y)] = 0
        # и слоты для статистической языковой модели y
        total[word_y] = 0

  # шаг 1. Expectation
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    total_stat = {} # статистика x

    # собираем предварительную статистику на основе данных x
    for word_x in X_train_tokens[i]:
      total_stat[word_x] = 0 # создаем слоты для подсчета статистики по каждому токену x
      for word_y in y_train_tokens[i]:
        # обновляем данные из t-table; увеличиваем значения при обнаружении совместной встречаемости
        total_stat[word_x] += t[(word_x, word_y)]

    # обновляем данные для P(x|y) и P(y)
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # подсчет условной вероятности совпадений в корпусе: равномерное распределение / частотность x
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        # подсчет статистической информации y: равномерное распределение / частотность x
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]

  # шаг 2. Maximization
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # обновляем t-table: вероятность совпадения в корпусе / вероятность информации y
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]

for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | with   ->   außerordentlich | Вероятность: 0.022
Соответствие | with   ->   begrüßen | Вероятность: 0.002
Соответствие | the   ->   Subsidiarität | Вероятность: 0.0
Соответствие | the   ->   außerordentlich | Вероятность: 0.0
Соответствие | the   ->   begrüßen | Вероятность: 0.0
Соответствие | principle   ->   All | Вероятность: 0.024
Соответствие | principle   ->   dies | Вероятность: 0.003
Соответствие | principle   ->   ist | Вероятность: 0.0
Соответствие | principle   ->   im | Вероятность: 0.0
Соответствие | principle   ->   Sinne | Вероятность: 0.003
Соответствие | principle   ->   der | Вероятность: 0.0
Соответствие | principle   ->   Subsidiarität | Вероятность: 0.144
Соответствие | principle   ->   außerordentlich | Вероятность: 0.144
Соответствие | principle   ->   zu | Вероятность: 0.0
Соответствие | principle   ->   begrüßen | Вероятность: 0.012
Соответствие | subsidiarity   ->   All | Вероят

**Биграмная модель**

In [ ]:
# для обучения модели объединим 2 выборки
tokens = ' '.join(german).split()

# хранилище для биграмм
bigram_model = defaultdict(list)

# собираем все попарные совпадения
for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)

print(bigram_model)

def decoder(model, steps=5):
  # инициализация случайного токена
  current_word = random.choice(tokens)
  generated_sentence = current_word

  for step in range(steps):
    # пошаговая генерация
    print('Шаг', step+1)
    next_word_options = model[current_word]
    print(f'Правдоподобные варианты продолжения для токена {current_word}:', next_word_options)

    current_word = random.choice(next_word_options)
    generated_sentence += ' '
    generated_sentence += current_word
    print('Промежуточный результат:', generated_sentence)
    print()
  print('Результат:', generated_sentence)

decoder(bigram_model)

defaultdict(<class 'list'>, {'Wiederaufnahme': ['der'], 'der': ['Sitzungsperiode', 'Unzulässigkeit', 'relativen', 'Position', 'Fall', 'Staatsanwalt', 'Arbeitsschutzausschuß', 'Fall', 'dortigen', 'großen', 'EU-Mitgliedstaaten', 'EU', 'Lage', 'PSE-Fraktion', 'Konferenz', 'Präsidenten', 'Konferenz', 'Präsidenten', 'Beschluß', 'Sozialistischen', 'PPE-DE-Fraktion', 'Tagesordnung', 'Fraktion', 'Tagesordnung', 'gegen', 'Werbetrommel', 'Abstimmung', 'Sitzung', 'Subsidiarität', 'ersten', 'Verkehrssicherheit', 'Prüfungsvorschriften', 'Kommission', 'Straße', 'Anwendung', 'technischen', 'Transportbehälter', 'Kommission', 'morgigen', 'Kommission', 'Zusätzlichkeit', 'Kommission', 'Wirtschaftsstruktur', 'Sozialwirtschaft', 'Arbeitsplätze', 'Union', 'fragwürdigen', 'Konservativen', 'entsprechenden', 'Leitlinien', 'Erarbeitung', 'Leitlinien', 'Evaluierung', 'Ergebnisse', 'Bericht', 'Städte', 'nationalen', 'regionalen', 'Mittel', 'Politik', 'Mittel', 'Mittelauszahlung', 'Mitgliedstaaten', 'Leitlinien', 

# **Оценка результатов**

In [ ]:
# сортировка t-table по убыванию правдоподобия
sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def translate(token):
  for element in sorted_t:
    if element[0][1] == token:
      # поиск совпадений в t-table
      return element[0][0]
  return token

for sentence in y_test_tokens:
  print("Оригинальное предложение:", ' '.join(sentence))
  translation = []
  for token in sentence:
    translation.append(translate(token))
  print("Перевод:", ' '.join(translation))

Оригинальное предложение: Doch die Positionen sind deutlich geworden und werden ins Protokoll aufgenommen .
Перевод: helped the reflect is loud geworden and , ins Minutes aufgenommen .
Оригинальное предложение: ( Das Parlament lehnt den Antrag mit 164 Ja-Stimmen , 166 Nein-Stimmen und 7 Enthaltungen ab . )
Перевод: ) This Parliament rejected the this , 164 Ja-Stimmen , 166 Nein-Stimmen and , Enthaltungen on . )
Оригинальное предложение: Es sollte nicht darum gehen , neue bürokratische Hindernisse zu errichten .
Перевод: a , not must cities , was bürokratische Hindernisse . cities .
Оригинальное предложение: Dies zwingt das Europäische Parlament , den Herrn Kommissar und die Kommission zu entschlossenem strategischen Handeln .
Перевод: interest zwingt . weaken Parliament , the Mr Commissioner and the Commission . entschlossenem strategischen Handeln .
Оригинальное предложение: Wie die jüngsten Ereignisse zeigen , gibt es in diesem Bereich noch sehr viel zu tun .
Перевод: is the jüngsten

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

reference = [X_test_tokens[0], X_test_tokens[1]]
candidate = [translate(token) for token in y_test_tokens[0]]

bleu_score = corpus_bleu(reference, candidate)

print("BLEU Score:", bleu_score)

AssertionError: The number of hypotheses and their reference(s) should be the same 

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

references = X_test_tokens
candidates = []
for sentence in y_test_tokens:
    candidates.append([translate(token) for token in sentence])


bleu_score = corpus_bleu(references, candidates)

print("BLEU Score:", bleu_score)

BLEU Score: 1.1041495953423772e-231


In [ ]:
reference

[['Even',
  'so',
  ',',
  'I',
  'think',
  'the',
  'positions',
  'are',
  'quite',
  'clear',
  'and',
  'they',
  'shall',
  'be',
  'entered',
  'in',
  'the',
  'Minutes',
  '.'],
 ['(',
  'Parliament',
  'rejected',
  'the',
  'request',
  ',',
  'with',
  '164',
  'votes',
  'for',
  ',',
  '166',
  'votes',
  'against',
  'and',
  '7',
  'abstentions',
  ')']]

In [ ]:
candidate

['helped',
 'the',
 'reflect',
 'is',
 'loud',
 'geworden',
 'and',
 ',',
 'ins',
 'Minutes',
 'aufgenommen',
 '.']